# Проект - Защита персональных данных клиентов

## Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Задачи проекта
1.	Загрузите и изучите данные.
2.	Ответьте на вопрос и обоснуйте решение. 
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 
- a) Изменится. Приведите примеры матриц.
- b) Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3.	Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
4.	Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.


## Описание данных

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('max_columns', None) # показывать все столбцы df
pd.set_option('max_rows', None) # показывать все строки df

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random

### 1. Загрузите и изучите данные.

In [2]:
# загрузка данных и вывод первых пяти строк
try:
    data = pd.read_csv("/datainsurance.csv")
except:
    data = pd.read_csv("insurance.csv")
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
# вывод последних пяти строк
data.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [4]:
# общая информация о df
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
# статистическая информация по данным 
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
# доп проверка на пропуски
data.isna().mean()

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

In [7]:
# проверка на явные дубликаты между строками
data.duplicated().sum()

153

In [8]:
# подготовка матрицы признаков и вектора целевых значений (таргета) 
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [9]:
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


**Выводы, информация п.1**
- загружены, изучены данные, выведена информацию о пяти первых и последних строках для оценки визуально
- выведена общая информация по df
- выведена информация по количеству пропусков в признаках
- сделана проверка на явные дубликаты по объектам 

информация по df:
- 5000 записей
- 5 колонок
- данные по явным дубликатам, т.к. и признаки и значения признаков достаточно типичные, то наиболее вероятно, что это совпали данные у разных объектов - удалять, изменять не будем

### пояснение по LaTeX

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### 2. Ответьте на вопрос и обоснуйте решение. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.) 
- a) Изменится. Приведите примеры матриц. 
- b) Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменяться

**Обоснование:** 

формула обучение весов модели: $ w = (X^TX)^{-1}X^Ty $

домножим матрицу весов X на матрицу P т.е. возьмем матрицу XP, wp - это веса с учетом матрицы XP: 

$ wp = ((XP)^T(XP))^{-1}(XP)^Ty $

применем свойства  $(AB)^T = B^TA^T$  и раскроем скобки внутри выражения линейной регрессии: 

$wp = ((P)^TX^TXP)^{-1}(P)^TX^Ty$ 

применем свойство $(AB)^{-1} = B^{-1}A^{-1}$ и раскроем скобки:

$ wp =  (P)^{-1} * (X^TX)^{-1} * (P^T)^{-1} * (P)^T * X^Ty $      

по свойству $ AA^{-1} = E $ сократим   $ (P^T)^{-1}(P)^T $ и получим: 

$ wp = (P)^{-1}(X^TX)^{-1}X^Ty $

используя $ w = (X^TX)^{-1}X^Ty $ заменим  $ (X^TX)^{-1}X^Ty $:

$ wp = (P)^{-1}w $:

т.к. веса связаны, то качество модели не изменится

подставим в формулу предсказания  $ a = Xw $ вместо X новую матрицу XP, а вместо w новый вектор весов $ (𝑃)−1𝑤 $ и применем свойство $ AA^{-1} = E $,  получим:

$ ap = XP(P)^{-1}w = Xw = a $





**проверка вывода на модели со случайной обратимой матрицой**

In [10]:
# генерируем случайную матрицу
random_matrix = np.random.randint(1, 10, size=(4, 4))
random_matrix

array([[4, 9, 9, 8],
       [7, 9, 3, 1],
       [3, 6, 7, 7],
       [6, 4, 4, 1]])

In [11]:
# проверка созданной случайной матрицы на обратимость
inv_random_matrix = np.linalg.inv(random_matrix)
inv_random_matrix 

array([[-0.55458515,  0.10043668,  0.60262009,  0.11790393],
       [ 0.26200873,  0.11790393, -0.29257642, -0.16593886],
       [ 0.75545852, -0.31004367, -0.86026201,  0.28820961],
       [-0.74235808,  0.16593886,  0.99563319, -0.19650655]])

In [12]:
# умножим признаки на обратимую матрицу
features_dot_rm = features @ random_matrix
features_dot_rm.head()

,0,1,2,3
0,149097.0,297982.0,347336.0,347250.0
1,114328.0,228418.0,266142.0,266047.0
2,63203.0,126261.0,147087.0,147029.0
3,125259.0,250397.0,291971.0,291923.0
4,78500.0,156861.0,182793.0,182736.0


In [13]:
# обучение, предсказание и расчет метрики R2 на фактических значениях features
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


In [14]:
# обучение, предсказание и расчет метрики R2 на features_dot_rm (умноженые признаки на обратимую матрицу)
model_2 = LinearRegression()
model_2.fit(features_dot_rm, target)
predictions_2 = model_2.predict(features_dot_rm)
print(r2_score(target, predictions_2))

0.42494550286667776


**Выводы, информация п.2**
- при умножении признаков на обратимую матрицу, качество линейной регрессии не изменяется

проверка:
- аналитический вывод

- расчет метрики R2 на признаках без и с умножением на обратимую матрицу, метрика R2 одинакова

### 3. Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм** -  признаки  умножим на случайную квадратную матрицу P, которая д.б.обратимой.  

Зашифрованые данные можем восстановить с помощью умножения преобразованной матрицы на обратную к P.
- $ XP = Xp $  
- $ Xp P^{-1} = XP * P^{-1} = X $

**Обоснование**: в п.2. ранее показали, что такое преобразрвание не влияет на качество предсказания.

### 4. Проверка алгоритма

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [15]:
# разделим данные на тренировочные и тестовые
features_train, features_test, target_train, target_test = train_test_split(
    features, target, random_state=12345, shuffle=True, test_size=0.25)

In [16]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('Значение метрики R2 на не зашифрованых данных:',r2_score(target_test, predictions))

Значение метрики R2 на не зашифрованых данных: 0.43522757127025635


**Проверка качества модели на зашифрованных данных**

In [17]:
# случайная матрица
random.seed(11)
random_matrix = np.random.randint(1, 10, size=(4, 4))
random_matrix

array([[7, 7, 5, 2],
       [3, 1, 2, 7],
       [1, 8, 2, 8],
       [2, 5, 2, 4]])

In [18]:
# проверка на обратимость
inverse_random_matrix = np.linalg.inv(random_matrix)
inverse_random_matrix

array([[ 7.14285714e+00,  5.71428571e-01,  1.38571429e+01,
        -3.22857143e+01],
       [ 2.00000000e+00, -1.58603289e-16,  4.00000000e+00,
        -9.00000000e+00],
       [-1.27142857e+01, -8.57142857e-01, -2.52857143e+01,
         5.84285714e+01],
       [ 2.85714286e-01,  1.42857143e-01,  7.14285714e-01,
        -1.57142857e+00]])

In [19]:
# признаки для обучения и теста на зашифрованных данных
features_train = features_train@random_matrix
features_test = features_test@random_matrix

In [20]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('Значение метрики R2 на зашифрованых данных:', r2_score(target_test, predictions))

Значение метрики R2 на зашифрованых данных: 0.435227571271228


**Выводы, информация п.4.**
- для изменения персональных данных применено умножение на случайную квадратную матрицу, 
  которая д.б.обратимой;
- сделан расчет метрики R2 для не зашифрованых и для зашифрованных данных;
- метрика R2 не изменилась;

### 5. Проверка, что защифрованые данные через умножение на случайную матрицу можно восставоить

In [21]:
# шифруем данные - домнажаем признаки на случайную квадратную матрицу
Xp = features @ random_matrix

# расшифровываем данные - переводим обратно через домнажение на обратную матрицу
X = abs(round(Xp @ inverse_random_matrix))

In [22]:
# новый df с расшифрованными данными - первые пять строк
new_df = pd.DataFrame(X)
new_df.columns=features.columns
new_df[['Пол', 'Возраст','Члены семьи']] = new_df[['Пол', 'Возраст','Члены семьи']].astype('int64')
new_df.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0


In [23]:
# исходный df - первые пять строк
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
